# Save mds vector output with time series of all ADCP longitudes/depths

In [53]:
import os
import numpy as np
import matplotlib.pyplot as plt
from xmitgcm import open_mdsdataset, utils
from MITgcmutils import rdmds, wrmds
import xarray as xr
from datetime import datetime, timedelta
import pych.ecco as pe

In [55]:
llc = pe.read_mds('/workspace/grids/llc90',iters=None)

/workspace/xmitgcm/xmitgcm/mds_store.py:854: UserWarning: Couldn't find available_diagnostics.log in /workspace/grids/llc90 or /workspace/grids/llc90. Using default version.
  "in %s or %s. Using default version." % (data_dir, grid_dir))


In [48]:
def get_single_timeseries(lon,klev):
    lonoffset = longs.index(lon)
    kgen = lonoffset*22 + klev + 1
    return np.squeeze(rdmds(f'../mdsfiles/uvel_all_lons/euc_out.{kgen:03}.0000000001',rec=np.arange(14478)))

In [49]:
def get_single_longitude(lon):
    all_depths = []
    for k in range(22):
        all_depths.append(get_single_timeseries(lon,k))
    return np.array(all_depths).T

In [50]:
def get_all_longitudes(londict):
    all_lons = []
    for key,val in londict.items():
        arr = get_single_longitude(val)
        print(arr.shape)
        all_lons.append(arr)
        
    return np.stack(all_lons,axis=2)

In [51]:
test = get_all_longitudes(lond)

(14478, 22)
(14478, 22)
(14478, 22)
(14478, 22)
(14478, 22)
(14478, 22)


In [76]:
llc['Z']

<xarray.DataArray 'Z' (k: 50)>
array([-5.000000e+00, -1.500000e+01, -2.500000e+01, -3.500000e+01,
       -4.500000e+01, -5.500000e+01, -6.500000e+01, -7.500500e+01,
       -8.502500e+01, -9.509500e+01, -1.053100e+02, -1.158700e+02,
       -1.271500e+02, -1.397400e+02, -1.544700e+02, -1.724000e+02,
       -1.947350e+02, -2.227100e+02, -2.574700e+02, -2.999300e+02,
       -3.506800e+02, -4.099300e+02, -4.774700e+02, -5.527100e+02,
       -6.347350e+02, -7.224000e+02, -8.144700e+02, -9.097400e+02,
       -1.007155e+03, -1.105905e+03, -1.205535e+03, -1.306205e+03,
       -1.409150e+03, -1.517095e+03, -1.634175e+03, -1.765135e+03,
       -1.914150e+03, -2.084035e+03, -2.276225e+03, -2.491250e+03,
       -2.729250e+03, -2.990250e+03, -3.274250e+03, -3.581250e+03,
       -3.911250e+03, -4.264250e+03, -4.640250e+03, -5.039250e+03,
       -5.461250e+03, -5.906250e+03])
Coordinates:
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48 49
    Z        (k) >f8 -5.0 -15.0 -25.0 -35.0 ... -5.039e+03 -5.461e+03 -5.906e+03
    drF      (k) >f8 10.0 10.0 10.0 10.0 10.0 ... 364.5 387.5 410.5 433.5 456.5
    PHrefC   (k) >f4 49.05 147.15 245.25 ... 49435.043 53574.863 57940.312
Attributes:
    standard_name:  depth
    long_name:      vertical coordinate of cell center
    units:          m
    positive:       down

In [82]:
def get_uvel(arr=None):
    
    lonstrs = ['110W','140W','170W','165E','156E','147E']
    longs = [-110,-140,-170,165,156,147]
    lond = dict(zip(lonstrs,longs))
    arr = get_all_longitudes(lond) if arr is None else arr
    
    ntime = arr.shape[0]
    ecco_time = np.array([np.datetime64(x) for x in datetime(1992,1,1,12)+(np.arange(ntime)+1)*timedelta(hours=1)])
    time = xr.DataArray(ecco_time,coords={'time':ecco_time},dims=('time',),name='time')
    longitude = xr.DataArray(np.array(longs),coords={'lon':np.array(longs)},dims=('lon',),name='lon')
    z = llc['Z'][:22].copy().swap_dims({'k':'Z'})
    depth = -z.rename({'Z':'depth'})
    depth.attrs['long_name'] = 'Depth'
    depth.attrs['units'] = 'm'
    
    uvel = xr.DataArray(arr,coords={'time':time,'depth':depth,'lon':longitude},dims=('time','depth','lon'),
                        attrs={'descrption':'Hourly Zonal Velocity',
                               'units':'m/s'})
    
    return uvel

In [83]:
uvel = get_uvel(test)

In [85]:
fname = '../ncfiles/eccov4r4_uvel_short.nc'

In [86]:
uvel.to_netcdf(fname)